# 信号空间
```{epigraph}
道可道非常道，名可名非常名。无名天地之始，有名万物之母。故常无欲以观其妙，常有欲以观其徼。此两者，同出而异名，同谓之玄，玄之又玄，众妙之门。

--老子
```

万千世界起源于基础的规律与法则。这些规律与法则是世界与生俱有的，不是能够被人们所定义的。天地起源于“无”，而“有”产生了万物。

因此我们应该客观地去观察其细节，而这种客观性是在主观的结构观察中进行。

有与无、客观与主观就是一种不可言的基础法则，非常玄妙，是我们认识世界，以及行事的门道。

## 分析流程之结构模型
信号处理就是对一个给定的信号，应用一系列的分析方法，进行分析。虽然分析的方案与细节因项目的性质与目的而异，但是分析流程在结构上有一定的规律。这里我们希望能回答以下的问题：
- 结构的范畴。
- 分析过程的共性。

### 范畴
#### 算法范畴
算法分四类：全局分析，局部分析，自适应分析和自学习性分析。全局分析和局部分析共享基础分析算法。总体结构归纳如下：
- 基础分析方法：傅里叶分析（尤其是余弦波和正弦波分析），卷积，小波分析等。
- 局部分析：对局部信号两端有两类不同的处理
  - 直接截取：对应与block transforms
  - 光滑延伸：对应与lap transforms
- 自适应分析：建立在多层次局部分析的基础上，从不同合法的组合中寻找最佳组合。
- 自学习性：建立在多层局部分析的基础上，也有可能依赖于自适应分析。

#### 信号范畴
对信号的范畴的界定归结为有限化和离散化。
- *时间域*：我们定义自然数$\mathbb{N}$为时间域的空间。也就是说进入这个模型的型号都被认为其时间域是$\mathbb N$，而且是按序排列。至于时间域的真实意义不在这个模型之内。
- *值域*：原始信号由一列数组构成，数组的元素为复数，这个我们称之为*0-空间*。为了满足局部分析，我们还需要引入两个空间，都是序列，区分在于序列的元素。对应与block transform，元素为数组，代表所截取的一段信号，这个空间我们称之为*1-空间*。因此，*1-空间*数组序列构成。对应与lap transforms，序列的元素是一个三体，三体中的每个子元素是一个数组，三体中的左元素是左边延伸的数组，右边是右边延伸的数组，中间则是由直接截取的局部信号。也就是说，这个空间是由三体序列构成，我们称之为*3-空间*。

### 分析过程的共性
#### 特征：
- 字面量数组是基础的分析单元，这个基础单元就是*0-空间*。分析算法最大的特点就是用基础分析方法反复应用在以*0-空间*为数据类型的元素上。
- 类一局部分析：这对应block transform。将类型为0-空间的原始信号截取$n$段，生成1-空间，空间内每个元素以0-空间为类型。遍历1-空间的元素，在遍历的过程中，将基础分析方法应用于每个元素，方法的输出也是0-空间类。
- 全局分析：我们把全局分析看成局部分析的特例。就是1-空间只含有一个以0-空间为类型的元素。
- 类二局部分析：这类分析对应lap transforms。Lap transform的单元函数（也就是作用于一个局部信号的函数）的输入与输出都是三体0-空间：（0-空间，0-空间，0-空间)。Lap transform就是运用单元函数遍历3-空间。

#### 容器（containers）与环境（contexts）
容器和环境的概念是对循环操作过程进行建模而产生。一个循环操作过程可以分解为三个模块：载体，基本元素，对基本元素的操作。容器就是载体，它是基本元素的一个集合，重复的操作是将操作函数作用于基础元素从而产生了新的基础元素。通过遍历我们获得了新的基本元素的集合，也就是新的容器。而这种操作行为我们看成给容器赋予了一个工作环境。

不同的性质的行为决定了不同的环境。行为的性质可以分为同构性与异构性。对多重循环而言，又可以分为独立性与依赖性。

##### 同构与异构
循环的操作就是将一个容器映射到另外一个容器，如果这两个容器是同一个结构，我们称之为同构，否则就是异构。比如函数`float`就是将基础元素为整数（类型为`int`）映射到相应的浮点数（类型为`float`）。一个`list`含有整数的容器在函数`float`的循环操作下变成了含有浮点数的`list`容器，这是一个同构操作。因为容器的结构没有变化，都是含有同样个数的`list`。如果这个操作函数不是`float`，而是将一个整数变成两个整数，那么这个循环操作行为就不是同构的，因为新的容器跟原来的容器含有不同的个数的基础元素。如果这个操作函数是将一个整数变成含有两个整数的一个`list`，那么这个循环操作又是同构的。

##### 独立与依赖
在多重循环中，有三个不同的行为，独立性，依赖性和嵌套性。
- 独立性是指每个循环可以独立进行。
- 依赖性是指当期循环依赖于上一个循环的结果。
- 嵌套式是指多重循环以嵌套的形式进行，也就是上一个循环要依赖于子循环结束才能进行下一个对基础元素的操作。

除了嵌套式外，函数式程式对这些不同环境有明确的定义。对同构，依赖性多循环，Functor是主要工具，Monad也可以胜任。对同构，独立多循环，Applicative是主要工具，Functor, Monad也是可以胜任。对异构，依赖性多循环，Monad是主要工具。

如何对嵌套式进行建模是一个研究课题。我们在自适应性算法实现的时候，会碰到这样的课题，我们解决的方案是采用二叉树的数据结构结合迭代式环境来实现。

## 模型描述
从数学的角度，我们定义了三个信号空间：$\mathfrak{F}_0=\mathbb{C}^N$, $\mathfrak{F}_1=\{\mathbb{C}^M\}_{k=1}^K~$和$~\mathfrak{F_3}=\{(C^{M_e},C^M,C^{M_e})\}_{k=1}^K$，分别对应0-空间，1-空间和3-空间。

从范畴（category）学的角度，信号空间可以定义为$\mathfrak{C}=\{\mathfrak{F}_1,\mathfrak{F}_3\}$，即信号空间是范畴$\mathfrak C$，其中含有$\mathfrak{F}_1$与$\mathfrak{F}_3$两个对象（object）。时间域的信息隐含在对象的指针（index）中。一个具体的分析方法本质上就是信号空间$\mathfrak C$的态射（morphism）。我们考虑的态射有一下两类：
- 自同态（endomorphism）：$\mathfrak F_i\longrightarrow\mathfrak F_i,\quad i\in\{1,3\}$，大多数的算法都为自同态。
- 跨对象态射：$\mathfrak F_i\longrightarrow\mathfrak F_j,\quad i,j\in\{1,3\}$，lap transform的逆变换就是从$\mathfrak F_3$到$\mathfrak F_1$的跨对象态射。

在这个模型下，基础算法不是态射，因为它们是$\mathfrak F_i\rightarrow\mathfrak F_i,\quad i\in\{0,1\}$或$\mathfrak F_i\rightarrow\mathfrak F_j,\quad i,j=\{0,3\}$的映射。信号空间的态射却是由这些基础算法提升（lifting）而来。从这个角度上看，上面所阐述的一个环境，就是一种提升的方法。

这个模型勾画了我们业务逻辑的本质，但过于抽象，为了方便实现，我们再定义两个工作范畴：$\mathfrak C_i=\{\mathfrak F_i\},\quad i\in\{1,3\}$。这两个范畴在我们的程序实现上对应与上述的环境。一个具体的分析方法就是这两个范畴之间的映射（$\mathfrak C_i$到$\mathfrak C_i$或$\mathfrak C_i$到$\mathfrak C_j$的映射）。

### 模型定义
综上所述，信号空间模型可以定义为：
- 三个基础集合：$\mathfrak{F}_0=\mathbb{C}^N$, $\mathfrak{F}_1=\{\mathbb{C}^M\}_{k=1}^K~$和$~\mathfrak{F_3}=\{(C^{M_e},C^M,C^{M_e})\}_{k=1}^K$。
- 两个范畴：$\mathfrak C_1=\{\mathfrak F_1\}$和$\mathfrak C_3=\{\mathfrak F_3\}$。
- 时间域隐含在集合元素的指针中。
- 分析方法为两个范畴之间的态射。
- 基础算法（简称算法）为基础集合之间的映射。
- 态射由算法提升而来，而这种提升的手段是通过对两个信号范畴的实现而获得，每一个特定的实现就是一种特定的环境，或者说环境就是对信号范畴在程序体中的表达。

## 模型的实现
以上模型的实现涵括以下内容：
- 信号空间的数据类型
- 信号空间环境与态射（算法的提升）
- 算法
- 态射

### 基础信号空间的数据类型
数据类型用type aliases来定义。对基础信号空间而言，这样可以利用`list`的诸多功能，尤其是`Iterable`和`Sequence`的功能。

- `signum`: 定义了0-空间的元素数据类型为字面量`int`, `float`, `complex`，加入`str`是将信号空间拓广到文本分析，`None`与`tuple[()]`则是技术上对空集处理的可能需求。
- `sig_0`：`list[signum]`，对应0-空间。
- `sig_1`: `list[sig_0]`，对应1-空间。
- `sig_3`: `list[(sig_0,sig_0,sig_0)]`，对应3-空间。

In [1]:
#from Signal.Types import signum, sig_0, sig_1, sig_3

signum=int|float|complex|str|None|tuple[()]
sig_0=list[signum]
sig_1=list[sig_0]
sig_3=list[tuple[sig_0,sig_0,sig_0]]

### 信号环境
这是对信号范畴的实现。环境包括两大类，计算环境（computational contexts）和迭代环境（iterator contexts）。

计算环境有：
- `Sigor`: `ListFunctor`约束于0-空间，1-空间和3-空间。
- `Sigive`：`ListApplicative`约束于0-空间，1-空间和3-空间。
- `SigMonad`：`ListMonad`约束于0-空间，1-空间和3-空间。

迭代环境：
- `SigFoldable`：`ListFoldable`约束于0-空间，1-空间和3-空间。
- `SigTraversable`：`ListTraversable`约束于0-空间，1-空间和3-空间。

其中`SigTraversable`还没实现。还没实现的计算环境是`SigStateMonad`，一个成熟的用于有限状态机的环境。

由于python的函数式功能散布在语言的各个部分，`map`，`reduce`，`filter`，`iter`，`itertools`，`functools`, `operator`，根据我们的需求我们有系统地对`list`容器赋予了一系列的环境。这一系列的实现放在`Contexts`程序包里，主要的手段就是`generator comprehension`，`abc`和`typing`。

In [2]:
#from Signal.Contexts import Sigor, Sigive, SigMonad, SigLiftA2

from Contexts.Functor import ListFunctor
from Contexts.Applicative import ListApplicative, liftA2
from Contexts.Monad import ListMonad
from Contexts.Foldable import ListFoldable, foldl, foldr, scanl, scanr

from Signal.Types import sig_0, sig_1, sig_3

Sigor=ListFunctor[sig_0|sig_1|sig_3]
Sigive=ListApplicative[sig_0|sig_1|sig_3]
SigMonad=ListMonad[sig_0|sig_1|sig_3]
SigLiftA2=liftA2
SigFoldable=ListFoldable[sig_0|sig_1|sig_3]


#### `Sigor`: 信号范畴的Functor环境
`Sigor`的关键功能就是`fmap`，是算法的提升器。`Sigor`适合同构，依赖性多循环的操作。以下案例从态射的角度，就是将`float`和`str`两个算法提升到态射层面，把原来的`sig_0[int]`映射为`sig_0[float]`，然后映射成`sig_0[str]`。从循环的角度，完成了两个有依赖的循环。

In [3]:
a=sig_0([x for x in range(1,9)])
s=Sigor(a)
s.fmap(float).fmap(str)

['1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0']

`Sigor`还有一个很有用的函数`Functor.const(a,b)`和一个建立在这个函数之上的类方法`mrb`(map-replaced by，在Haskell语言中是算子`<$`)，其功能是将输入参数替换掉容器的每一个元素。`mrb`是通过`fmap.const`来实现，这是一个很有启发的技巧。下面的案例就是将原来的数组`[1,2,3,4,5,6,7,8]`中的每一个元素替换成`[1,2]`:

In [8]:
s.mrb([1,2])

[[1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2]]

等价于以下操作：

In [9]:
s.fmap(lambda _: [1,2])

[[1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2]]

#### `Sigive`：信号范畴的Applicative环境
`Sigive`在`Sigor`的基础上加上了`ap`类方法。`ap`的功能是同时提升输入参数中每个算法并且对自身进行映射，将结果依次合并到同一容器中。输入参数必须是一个`list`的单参数函数序列。`Sigive`适合同构，独立性多循环操作。

以下案例是将`float`和`str`同时应用于自身并依次将结果合并成`sig_0`容器：

In [10]:
s=Sigive(a)
s.ap([float,str])

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8']

以下两个案例演示`Sigive`也是`Sigor`:

In [11]:
s.fmap(float)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

In [12]:
s.mrb([3,4])

[[3, 4], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4], [3, 4]]

### 算法
#### 边界处理方法
#### 方法
边界处理方法包含零延伸，周期延伸和对称延伸三种。假设信号长度为$n$，延伸的长度为$m$，通常我们要求$m\leq n$，信号为$[x_1,x_2,\cdots,x_n]$。

顾名思义，零延伸就是对区间外用零值延伸。即左右延伸都是$[0,\cdots,0]$，延伸的数组长度为$m$。

周期延伸是将区间中的信号看成周期函数进行延伸，因此，左延伸数组为$[x_{n-m+1},\cdots,x_n]$，这是因为根据周期性，元素$x_1$往左跟元素$x_n$相邻。同理，右延伸数组为$[x_1,x_2,\cdots,x_m]$。

对称延伸是以端点为原点的对称延伸：左延伸数组为$[x_m,x_{m-1},\cdots,x_1]$, 右延伸数组为$[x_n,x_{n-1},\cdots,x_{n-m+1}]$

#### 程序设计
程序包括三个数据类型，三种算法函数。三种函数为`zero`，`periodic`，`symetric`分别与零延伸，周期延伸和对称延伸相对应。函数的接口：输入接口为`(n,v)`，其中`n`为延伸的长度，类型为`int`，`v`为被延伸的信号，类型为`sig_0`。函数要求`n`不大于信号的长度的正整数。输出接口为`Extensions`的数据类，这个数据类是一对`sig_0`，即`(sig_0,sig_0)`，其中左元素对应左延伸数组，右元素对应右延伸数组。为了方便使用，定义了另外两个数据类：
- `PaddingMethodNames`: 延伸方法名称，`Enum`数据类，其中的域值为`ZERO`,`PERIODIC`和`SYMETRIC`。
- `padding`: dictionary数据类，其中`key`为`PaddingMethoNames`，`value`为上面的延伸函数。

In [4]:
#from Signal.Padding import PaddingMethodNames, padding

from typing import NewType
from enum import Enum
from Signal.Types import sig_0

Extensions = NewType('Extensions',tuple[sig_0,sig_0])
PaddingMethodNames=Enum('PaddingMethod',['ZERO','PERIODIC','SYMETRIC'])

def zero(n:int,v:sig_0)->Extensions:
    if n<0 or n>len(v): raise ValueError("Illegal Input")
    else: return Extensions(([0.0]*n,[0.0]*n))

def periodic(n:int,v:sig_0)->Extensions:
    if n<0 or n>len(v): raise ValueError("Illegal Input")
    else: return Extensions((v[-n:],v[:n]))

def symetric(n:int,v:sig_0)->Extensions:
    if n<0 or n>len(v): raise ValueError("Illegal Input")
    else: return Extensions((v[:n][::-1],v[-n:][::-1]))

padding={name:method for (name,method) in zip(PaddingMethodNames,(zero,periodic,symetric))}

In [5]:
for i in PaddingMethodNames: print(i)

PaddingMethod.ZERO
PaddingMethod.PERIODIC
PaddingMethod.SYMETRIC


In [6]:
a=sig_0([x for x in range(1,9)])
foo = map((lambda x: padding[x](4,a)), PaddingMethodNames)
print('原信号: ',a)
for i,j in zip(PaddingMethodNames,foo): print(str(i)+":",j)

原信号:  [1, 2, 3, 4, 5, 6, 7, 8]
PaddingMethod.ZERO: ([0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0])
PaddingMethod.PERIODIC: ([5, 6, 7, 8], [1, 2, 3, 4])
PaddingMethod.SYMETRIC: ([4, 3, 2, 1], [8, 7, 6, 5])


In [7]:
#from Signal.Utils import ispower2

def ispower2(n:int)->bool:
    return (n & (n-1) == 0) and n != 0